# SCRAP INE Y CORREOS

En este notebook se desarrolla un código para la obtención de información masiva de las webs públicas de INE y CORREOS.

Para poder utilizarse deben cambiarse las rutas de entrada y salida de datos, así como hacer una revisión de los códigos HTML de los elementos de las webs debido a su naturaleza dinámica (puede cambiar de un día para otro el identificador de cualquiera de los elementos)

Se compone de varias partes:

1. Tratamiendo inicial de datos para el scraping posterior.
2. Scraping del código INE de una serie de poblaciones.
3. Scraping del código postal de una serie de poblaciones.
4. Post procesado. Aquí se relacionan los códigos INE con los CP de las poblaciones.

# LIBRERÍAS NECESARIAS

In [ ]:
import numpy
import pandas as pd
import unicodedata
import re
import os
from datetime import datetime
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from tqdm import tqdm

## TRATAMIENTO INICIAL DE DATOS

In [ ]:
df_unidades_poblacionales=pd.read_excel('ruta_1/DATOS POBLACIONES/DATOS_MUNICIPIOS.xlsx',sheet_name='Hoja5',dtype=str)
df_datos_municipios=pd.read_excel('ruta_1/POBLACION_INE_CP.xlsx',sheet_name='Hoja1')
df_errores=pd.read_excel('ruta_1/POBLACION_INE_CP.xlsx',sheet_name='Hoja2')
df_entidades=pd.read_excel('ruta_1/ENTIDADES.xlsx',sheet_name='Hoja1')
df_cp=pd.read_excel('ruta_1/DATOS POBLACIONES/ENTIDADES.xlsx',sheet_name='CP',dtype=str)

# Eliminar la primera columna y establecer la primera fila como los títulos de las columnas en df1
df_unidades_poblacionales = df_unidades_poblacionales.drop(df_unidades_poblacionales.columns[3], axis=1)
df_unidades_poblacionales = df_unidades_poblacionales.drop(df_unidades_poblacionales.columns[0], axis=1)
df_unidades_poblacionales = df_unidades_poblacionales.rename(columns=df_unidades_poblacionales.iloc[0])
df_unidades_poblacionales = df_unidades_poblacionales.drop(df_unidades_poblacionales.index[0])

# Eliminar la primera columna y establecer la primera fila como los títulos de las columnas en df2
df_datos_municipios = df_datos_municipios.drop(df_datos_municipios.columns[0], axis=1)
df_datos_municipios = df_datos_municipios.rename(columns=df_datos_municipios.iloc[0])
df_datos_municipios = df_datos_municipios.drop(df_datos_municipios.index[0])

# Definir función para quitar tildes y poner en minúscula
def quitar_tildes(texto):
    texto_sin_tildes = ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))
    return texto_sin_tildes.lower()

# Aplicar la función a las columnas "PROVINCIA" y "POBLACIÓN" del DataFrame
df_datos_municipios["PROVINCIA"] = df_datos_municipios["PROVINCIA"].apply(quitar_tildes)
df_datos_municipios["POBLACIÓN"] = df_datos_municipios["POBLACIÓN"].apply(quitar_tildes)
df_unidades_poblacionales["CITY1"]=df_unidades_poblacionales["CITY1"].apply(quitar_tildes)

df_entidades['_CODIGOINE'] = df_entidades['_CODIGOINE'].str.replace('_', '').astype(str)
# df_entidades = df_entidades.drop(['COD_PROV', 'PROVINCIA', 'TIPO', 'POBLACION', 'INEMUNI', 'HOJA_MTN25', 'LONGITUD_ETRS89', 'LATITUD_ETRS89', 'ORIGENCOOR', 'ALTITUD', 'ORIGENALTITUD', 'SUPRIMIDA_INE', 'DISCREPANTE_INE'], axis=1)
df_entidades = df_entidades.drop([ 'CODIGOINE'], axis=1)
df_entidades["NOMBRE"]=df_entidades["NOMBRE"].apply(quitar_tildes)
df_entidades['_CODIGOINE'] = df_entidades['_CODIGOINE'].str.slice(stop=5)
df_entidades = df_entidades.drop_duplicates(subset=['NOMBRE', '_CODIGOINE'])

df_entidades.to_excel('ruta_1/entidades_depuradas.xlsx')

df_errores = df_errores.drop(df_errores.columns[0], axis=1)
df_errores = df_errores.drop(df_errores.columns[0], axis=1)
df_errores = df_errores.rename(columns=df_errores.iloc[0])
df_errores = df_errores.drop(df_errores.index[0])
df_errores = df_errores.astype(str)

df_errores['POBLACIÓN / UD POBLACIONAL']=df_errores['POBLACIÓN / UD POBLACIONAL'].apply(quitar_tildes)
df_errores.drop_duplicates(inplace=True)

df_errores['CLAVE']=df_errores['POBLACIÓN / UD POBLACIONAL']+'_'+df_errores['INE_CP']

df_errores.drop_duplicates(subset=['CLAVE'], inplace=True)

df_errores.to_excel('ruta_1/SOL_ERRORES_POB.xlsx')

df_unidades_poblacionales.to_excel('ruta_1/referencia_NORMALIZADO.xlsx')

df_cp=df_cp.drop(['NOMBRE_COMUNIDAD', 'COD_PROVINCIA', 'COD_MUNICIPIO'],axis=1)
df_cp["NOMBRE_POBLACION"]=df_cp["NOMBRE_POBLACION"].apply(quitar_tildes)
df_cp["NOMBRE_MUNICIPIO"]=df_cp["NOMBRE_MUNICIPIO"].apply(quitar_tildes)
df_cp.to_excel('ruta_1/CP_depurados.xlsx')

# eliminamos los espacios adicionales al final de las cadenas en las columnas 'NOMBRE_POBLACION'
df_cp['NOMBRE_POBLACION'] = df_cp['NOMBRE_POBLACION'].str.strip()
df_entidades['NOMBRE_POBLACION'] = df_entidades['NOMBRE_POBLACION'].str.strip()

# renombramos la columna 'NOMBRE' a 'NOMBRE_POBLACION' en el dataframe df_entidades
df_entidades = df_entidades.rename(columns={'NOMBRE': 'NOMBRE_POBLACION'})

# realizamos el OUTER JOIN por la columna común 'NOMBRE_POBLACION'
df_resultado = df_cp.merge(df_entidades, how='outer', on='NOMBRE_POBLACION')
df_resultado3 = df_cp.merge(df_entidades, how='inner', on='NOMBRE_POBLACION')

df_resultado1 = df_cp.merge(df_entidades, how='outer', left_on='NOMBRE_POBLACION', right_on='NOMBRE')
df_resultado2= df_cp.merge(df_entidades, how='outer', left_on='NOMBRE_MUNICIPIO', right_on='NOMBRE')

df_entidades = df_entidades.rename(columns={'NOMBRE_POBLACION': 'NOMBRE_MUNICIPIO'})
df_resultado4=df_cp.merge(df_entidades, how='inner', on='NOMBRE_MUNICIPIO')

df_resultado1.to_excel('ruta_1/res1.xlsx')
df_resultado2.to_excel('ruta_1/res2.xlsx')

# concatenamos los dataframes df_resultado3 y df_resultado4
df_concatenado = pd.concat([df_resultado3, df_resultado4], ignore_index=True)

df_concatenado.to_excel('ruta_1/concatenado.xlsx')

df_unidades_poblacionales=df_unidades_poblacionales.rename(columns={'CITY1': 'NOMBRE_MUNICIPIO'})

referencia1 = pd.merge(df_unidades_poblacionales,df_concatenado, how='left', on='NOMBRE_MUNICIPIO')

df_unidades_poblacionales=df_unidades_poblacionales.rename(columns={'NOMBRE_MUNICIPIO': 'NOMBRE_POBLACION'})

referencia2 = pd.merge(df_unidades_poblacionales,df_concatenado, how='left', on='NOMBRE_POBLACION')

df_concatenado2=pd.concat([referencia1, referencia2], ignore_index=True)

df_concatenado3=pd.concat([df_concatenado, df_concatenado2], ignore_index=True)

df_concatenado3['POST_CODE1'] = df_concatenado3['POST_CODE1'].str.zfill(5)

df_concatenado3.to_excel('ruta_1/final.xlsx')

df_entidades.reset_index()
df_cp.reset_index()
posicion_cp = df_cp.loc[df_cp['NOMBRE_POBLACION'] == 'la albuera'].index[0]
posicion_entidades = df_entidades.loc[df_entidades['NOMBRE_POBLACION'] == 'la albuera'].index[0]

# comparamos las posiciones
if posicion_cp == posicion_entidades:
    print("La posición de 'la albuera' es la misma en ambos dataframes: ", posicion_cp)
else:
    print("Las posiciones de 'la albuera' son diferentes en cada dataframe.")
    print("Posición en df_cp: ", posicion_cp)
    print("Posición en df_entidades: ", posicion_entidades)


# extraemos la fila 3393 del dataframe df_cp
fila_3393 = df_cp.iloc[3392]

# mostramos la fila extraída
print(fila_3393)

valor_cp = df_cp.iloc[posicion_cp]['NOMBRE_POBLACION']
valor_entidades = df_entidades.iloc[posicion_entidades]['NOMBRE_POBLACION']

# comparamos los valores
if valor_cp == valor_entidades:
    print("El valor de NOMBRE_POBLACION es el mismo en ambas posiciones: ", valor_cp)
else:
    print("Los valores de NOMBRE_POBLACION son diferentes en cada posición.")
    print("Valor de NOMBRE_POBLACION en la posición ", posicion_cp, " en df_cp: ", valor_cp)
    print("Valor de NOMBRE_POBLACION en la posición ", posicion_entidades, " en df_entidades: ", valor_entidades)


#LEFT JOIN A LOS CP DE referencia PARA SACAR

df_cp['C.POSTAL'] = df_cp['C.POSTAL'].astype(str).str.zfill(5)

df_cp['COD_PROVINCIA'] = df_cp['COD_PROVINCIA'].astype(str).str.zfill(2)
df_cp['COD_MUNICIPIO'] = df_cp['COD_MUNICIPIO'].astype(str).str.zfill(4)

df_cp['COD_INE']=df_cp['COD_PROVINCIA']+df_cp['COD_MUNICIPIO']

df_cp = df_cp.rename(columns={'C.POSTAL': 'CP'})
df_unidades_poblacionales=df_unidades_poblacionales.rename(columns={'POST_CODE1': 'CP'})

CP_INE_referencia = pd.merge(df_unidades_poblacionales,df_cp, how='left', on='CP')

# Agrupa los datos por el valor de CP y concatena los valores de COD_INE separados por una coma
CP_INE_referencia_2=CP_INE_referencia.copy()

# Agrupa los datos por "CP" y crea una nueva Serie de pandas con todos los valores de "COD_INE" para cada grupo, eliminando los valores duplicados
cod_ine_por_cp = CP_INE_referencia_2.groupby('CP')['COD_INE'].apply(lambda x: ','.join(pd.unique(x).astype(str))).reset_index(name='COD_INE para cada CP')

# Combina la nueva columna con el DataFrame original
df_con_cod_ine = pd.merge(CP_INE_referencia_2, cod_ine_por_cp, on='CP')

# Elimina las filas duplicadas
df_con_cod_ine_sin_duplicados = df_con_cod_ine.drop_duplicates()


df_con_cod_ine.to_excel('ruta_1/df_con_cod_ine.xlsx')

df_datos_referencia=pd.read_excel('ruta_1/DATOS_INE.xlsx',sheet_name='Hoja4',dtype=str)
df_datos_ine=pd.read_excel('ruta_1/DATOS_INE.xlsx',sheet_name='Hoja2',dtype=str)

# Eliminar la primera columna y establecer la primera fila como los títulos de las columnas en datos_referencia
df_datos_referencia = df_datos_referencia.drop(df_datos_referencia.columns[0], axis=1)
df_datos_referencia = df_datos_referencia.rename(columns=df_datos_referencia.iloc[1])
df_datos_referencia = df_datos_referencia.drop(df_datos_referencia.index[0])

df_datos_ine = df_datos_ine.drop(df_datos_ine.columns[3], axis=1)
df_datos_ine = df_datos_ine.drop(df_datos_ine.columns[2], axis=1)
df_datos_ine = df_datos_ine.drop(df_datos_ine.columns[1], axis=1)
df_datos_ine = df_datos_ine.drop(df_datos_ine.columns[0], axis=1)

copia_referencia=df_datos_referencia.copy()
copia_datos_ine=df_datos_ine.copy()

# función para aplicar a cada valor de la columna CITY1
def transform_city(city):
    # expresión regular para buscar los artículos en el nombre de ciudad
    pattern = r'^(EL|LA|LOS|LAS)\s'
    # buscar el patrón en el nombre de ciudad
    match = re.search(pattern, city)
    if match:
        # si se encuentra el patrón, reemplazar el artículo con el artículo entre paréntesis
        article = match.group(1)
        city = re.sub(pattern, '', city)
        city = f'{city} ({article})'
    return city

# aplicar la función a la columna CITY1
df_datos_referencia['CITY1'] = df_datos_referencia['CITY1'].apply(lambda x: transform_city(x))


# función para aplicar a cada valor de la columna CITY1
def transform_city2(city):
    # expresión regular para buscar el artículo al final del nombre de ciudad después de una coma
    pattern_end = r',\s*(EL|LA|LOS|LAS)$'
    
    # buscar el patrón de artículo al final del nombre de ciudad después de una coma
    match_end = re.search(pattern_end, city)
    if match_end:
        # si se encuentra el patrón, reemplazar el artículo con el artículo entre paréntesis
        article = match_end.group(1)
        city = re.sub(pattern_end, '', city)
        city = f'{city} ({article})'
        return city
    
    # si no se encuentra ningún patrón, devolver el nombre de ciudad sin cambios
    return city

# aplicar la función a la columna CITY1
df_datos_referencia['CITY1'] = df_datos_referencia['CITY1'].apply(lambda x: transform_city2(x))

df_datos_referencia = df_datos_referencia.drop_duplicates(subset=['POST_CODE1', 'CITY1'])

## SCRAPING_INE

In [ ]:

# IMPORTACIÓN DEL EXCEL CON LOS DATOS DE CP POBLACIÓN Y COD INE

df_datos=pd.read_excel('ruta_1/CP_INE/CP_INE_py_faltan_9_20.xlsx',sheet_name='Hoja1',dtype=str)
print("Datos importados")
def find_longest_word(text):
    words = text.split()
    longest_word = max(words, key=len)
    return longest_word

df_datos['POBLACION'] = df_datos['CITY1'].apply(find_longest_word)
df_datos['POBLACION'] = df_datos['POBLACION'].str.replace("-", "")
df_datos['POBLACION'] = df_datos['POBLACION'].str.replace(",", "")

# CONEXIÓN A LA WEB DEL INE Y COMIENZO DEL BUCLE DE INTRODUCCIÓN DE CÓDIGOS Y OBTENCIÓN DE TABLAS

# driver = webdriver.Chrome("ruta_2/chromedriver_win32/chromedriver.exe")
driver = webdriver.Chrome()
driver.get('https://www.ine.es/nomen2/index.do')
driver.maximize_window() #Maximizo la pantalla
time.sleep(2)
print("Conectado con el INE")
# Hacer clic en el botón de aceptar cookies
aceptar_cookies = driver.find_element('id','aceptarCookie')
aceptar_cookies.click()

datos = []
print("Datos cargados")
i=1
for index, row in df_datos.iterrows():
    # Obtener el nombre clave de la población
    # index=0
    name = row['POBLACION']
    print(name)
    
    porc_completo=(i/len(df_datos))*100
    i=i+1
    print(f'Completado al {porc_completo} %')
    
    # Introducir el código de provincia y el código de municipio en los campos correspondientes
    campo_nombre = driver.find_element('id',"nombrePoblacion")
    campo_nombre.clear()#Limpio el campo donde introduzco la referencia
    campo_nombre.send_keys(name)

    # time.sleep(2)
    # Pulsar la tecla ENTER para enviar los datos
    campo_nombre.send_keys(Keys.ENTER) #Pulso intro en el campo
    
    # Datos tabla
    try:                    
            # Esperar a que la tabla cargue
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "datos")))
            
            # Obtener la tabla
            tabla = driver.find_element(By.CLASS_NAME, "datos")
            
            # Obtener la cabecera de la tabla
            cabecera = tabla.find_element(By.CLASS_NAME, "cabtab")
            
            # Obtener los nombres de las columnas de la tabla
            provincia = driver.find_element('id', "p").text
            municipio = driver.find_element('id', "m").text
            unidad_poblacional = driver.find_element('id', "u").text
            
            # Obtener los datos de la tabla
            cuerpo = tabla.find_element(By.TAG_NAME, "tbody")
            filas = cuerpo.find_elements(By.TAG_NAME, "tr")
            
            # Iterar por las filas de la tabla
            for fila in filas[2:]:
                # print(fila)
                # Encontrar los elementos th y td de la fila
                ths = fila.find_elements(By.TAG_NAME,'th')
                # print(ths)
                # tds = fila.find_elements_by_tag_name('td')
                
                # Extraer el texto de los elementos th y td y almacenarlo en variables
                codigo_provincia = ths[0].text.split()[0] # Código de provincia
                provincia = ' '.join(ths[0].text.split()[1:]) # Nombre de la provincia
                codigo_municipio = ths[1].text.split()[0] # Código de municipio
                municipio = ' '.join(ths[1].text.split()[1:]) # Nombre del municipio
                codigo_up = ths[2].text.split()[0] # Código de unidad poblacional
                up = ' '.join(ths[2].text.split()[1:]) # Nombre de la unidad poblacional
                
                # Imprimir los valores
                # print(codigo_provincia, provincia, codigo_municipio, municipio, codigo_up, up)#, poblacion_total.text, hombres.text, mujeres.text)
                datos.append({
                              'Cod provincia':codigo_provincia,
                              'Provincia': provincia, 
                              'Cod municipio': codigo_municipio,
                              'Municipio': municipio, 
                              'Codigo ud pob': codigo_up,
                              'Ud pob': up})
    except:
        # En caso de que no se encuentre la tabla, añadir una fila con valores nulos
                datos.append({
                              'Cod provincia':None,
                              'Provincia': None, 
                              'Cod municipio': None,
                              'Municipio': None, 
                              'Codigo ud pob': None,
                              'Ud pob': None})
    driver.get('https://www.ine.es/nomen2/index.do ')
# Cerrar el driver
driver.quit()

# Convertir la lista de datos en un dataframe
df_resultado = pd.DataFrame(datos)

# Imprimir el resultado
print(df_resultado)

# EXPORTACIÓN DE RESULTADOS
df_resultado.to_excel('ruta_1/CP_INE/datos_ine_nuevos_2.xlsx')

## SCRAPING_CORREOS

In [ ]:
# IMPORTACIÓN DEL EXCEL CON LOS DATOS DE CP POBLACIÓN Y COD INE
 
df_datos_7=pd.read_excel('ruta_1/BASE DATOS INE CP DEFINITIVO_AUX.xlsx',sheet_name='FALTAN CORREOS',dtype=str)
print("Datos importados")
 
# CONEXIÓN A LA WEB DEL INE Y COMIENZO DEL BUCLE DE INTRODUCCIÓN DE CÓDIGOS Y OBTENCIÓN DE TABLAS
 
driver = webdriver.Chrome(executable_path=r"ruta_2\chromedriver_win32\chromedriver.exe")
driver.get('https://www.correos.es/es/es/herramientas/codigos-postales/detalle')
driver.maximize_window() #Maximizo la pantalla
time.sleep(2)
print("Conectado con el Correos")
# Hacer clic en el botón de aceptar cookies
x_path_aceptar_cookies = "//button[contains(@class,'sc-correos-ui-button') and contains(@aria-label,'Aceptar todas las cookies')]"
# Esperar hasta que el botón esté visible
aceptar_cookies = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, x_path_aceptar_cookies)))
# Hacer clic en el botón
aceptar_cookies.click()

# Crear dataframe general vacío
df_general = pd.DataFrame()

print("Datos cargados")
i=1

for index, row in df_datos_7.iterrows():
    # Obtener el código de provincia y el código de municipio de la fila actual
    index=0
    cp = row['CP']
    print(cp)
    i=i+1
    porc_completo=(i/len(df_datos_7))*100
    print(f'Completado al {porc_completo} %')
    
    # Introducir el código de provincia y el código de municipio en los campos correspondientes
    campo_cp = driver.find_element(By.CSS_SELECTOR,"input[id*='correos-ui-input']")
    campo_cp.clear()#Limpio el campo donde introduzco la referencia
    campo_cp.send_keys(cp)
    
    # Pulsar la tecla ENTER para enviar los datos
    campo_cp.send_keys(Keys.ENTER) #Pulso intro en el campo

    # Datos tabla
    try:                 
            # Esperar a que se muestren los resultados de la búsqueda
            elemento_resultados = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.sc-correos-ui-list-cp-results-h')))
            
            # Obtener la cadena que contiene los datos
            literals = driver.find_element(By.CSS_SELECTOR,'.sc-correos-ui-list-cp-results-h').get_attribute('literals')
            # Convertir la cadena a un objeto JSON
            literals_json = json.loads(literals.replace('&quot;', '"'))
            
            postal_codes = []
            provincias = []
            localidades = []
            
            # Iterar sobre la lista de literales y extraer los datos necesarios
            for literal in literals_json:
                postalCodeNumber = literal['postalCodeNumber']
                provincia = literal['listDescription'][0]['definition']
                localidad = literal['listDescription'][1]['definition']
                
                postal_codes.append(postalCodeNumber)
                provincias.append(provincia)
                localidades.append(localidad)
            df = pd.DataFrame({'postal_code': postal_codes, 'provincia': provincias, 'localidad': localidades})
    except:
        # Si no se encuentra la tabla, crear un dataframe con valores nulos
        df = pd.DataFrame({'postal_code': [None], 'provincia': [None], 'localidad': [None]})
        print(f'Error en {cp}')
    
    # Añadir el dataframe al dataframe general
    df_general = pd.concat([df_general, df], ignore_index=True)

# Cerrar el driver
driver.quit()

# Imprimir el resultado
print(df_general)

df_general.to_excel('ruta_1/correos_7.xlsx')

## POST PROCESADO

Una vez se tienen los datos masivos descargados se procesan para buscar la correspondencia del código INE de cada población con el código postal extraído de la web de correos.

In [ ]:
faltan = pd.read_excel('ruta_1/CP_INE/CIS_2021_20230407.XLSX',
                       sheet_name='Faltan',
                       dtype={'POST_CODE1': str, 'CITY1': str})

base_ine = pd.read_excel('ruta_1/CP_INE/CIS_2021_20230407.XLSX',
                         sheet_name='BASE DATOS INE',
                         dtype={'COD_INE': str, 'Ud pob': str})

# Supongamos que tus dataframes se llaman df1 y df2
df1 = faltan
df2 = base_ine

# Vamos a convertir los valores de las columnas POST_CODE y COD_INE a string
df1['POST_CODE1'] = df1['POST_CODE1'].astype(str)
df2['COD_INE'] = df2['COD_INE'].astype(str)

# Creamos una función para buscar la coincidencia de los dos primeros dígitos y un 80% de similitud en las ciudades
def find_cod_ine(row):
    post_code = row['POST_CODE1']
    city1 = row['CITY1']
    
    # Filtramos el df2 por los dos primeros dígitos
    df2_filtered = df2[df2['COD_INE'].str.startswith(post_code[:2])]
    
    # Si no hay coincidencias en los dos primeros dígitos, retornamos NaN
    if df2_filtered.empty:
        return float('nan')
    
    for index, row in df2_filtered.iterrows():
        if fuzz.ratio(city1.lower(), row['Ud pob'].lower()) >= 100:
            return row['COD_INE'], row['Ud pob']

    return float('nan'), float('nan')

# Aplicamos la función a cada fila de df1
df1[['COD_INE_MATCH', 'UD_POB_MATCH']] = df1.apply(
    lambda row: pd.Series(find_cod_ine(row)), axis=1)

df1_terminado=df1.loc[df1['COD_INE_MATCH'].notna()]
df1=df1[df1['COD_INE_MATCH'].isna()]

len(df1)
df1_terminado.to_excel('ruta_1/CP_INE/CP_INE_py_2.xlsx')

# Creamos una función para buscar la coincidencia de los dos primeros dígitos y un 80% de similitud en las ciudades
def find_cod_ine(row):
    post_code = row['POST_CODE1']
    city1 = row['CITY1']
    
    # Filtramos el df2 por los dos primeros dígitos
    df2_filtered = df2[df2['COD_INE'].str.startswith(post_code[:2])]
    
    # Si no hay coincidencias en los dos primeros dígitos, retornamos NaN
    if df2_filtered.empty:
        return float('nan')
    
    for index, row in df2_filtered.iterrows():
        if fuzz.ratio(city1.lower(), row['Ud pob'].lower()) >= 90:
            return row['COD_INE'], row['Ud pob']

    return float('nan'), float('nan')

# Aplicamos la función a cada fila de df1
df1[['COD_INE_MATCH', 'UD_POB_MATCH']] = df1.apply(
    lambda row: pd.Series(find_cod_ine(row)), axis=1)

df1_terminado2=df1.loc[df1['COD_INE_MATCH'].notna()]
df1=df1[df1['COD_INE_MATCH'].isna()]

len(df1)
df1_terminado2.to_excel('ruta_1/CP_INE/CP_INE_py_3.xlsx')

df1.to_excel('ruta_1/CP_INE/CP_INE_py_faltan_9_20.xlsx')

# Creamos una función para buscar la coincidencia de los dos primeros dígitos y un 80% de similitud en las ciudades
def find_cod_ine(row):
    post_code = row['POST_CODE1']
    city1 = row['CITY1']
    
    # Filtramos el df2 por los dos primeros dígitos
    df2_filtered = df2[df2['COD_INE'].str.startswith(post_code[:2])]
    
    # Si no hay coincidencias en los dos primeros dígitos, retornamos NaN
    if df2_filtered.empty:
        return float('nan')
    
    for index, row in df2_filtered.iterrows():
        if fuzz.ratio(city1.lower(), row['Ud pob'].lower()) >= 80:
            return row['COD_INE'], row['Ud pob']

    return float('nan'), float('nan')

# Aplicamos la función a cada fila de df1
df1[['COD_INE_MATCH', 'UD_POB_MATCH']] = df1.apply(
    lambda row: pd.Series(find_cod_ine(row)), axis=1)

df1_terminado3=df1.loc[df1['COD_INE_MATCH'].notna()]
len(df1_terminado3)

df1=df1[df1['COD_INE_MATCH'].isna()]
len(df1)

df1_terminado3.to_excel('ruta_1/CP_INE/CP_INE_py_4.xlsx')

faltan = pd.read_excel('ruta_1/CP_INE/CP_INE_py_faltan_9_20.xlsx',
                       sheet_name='Sheet1',
                       dtype={'POST_CODE1': str, 'CITY1': str})

faltan2 = pd.read_excel('ruta_1/CP_INE/CP_INE_py_faltan_9_20.xlsx',
                       sheet_name='Hoja1',
                       dtype={'POST_CODE1': str, 'CITY1': str})

base_ine = pd.read_excel('ruta_1/CP_INE/datos_ine_nuevos_1.xlsx',
                         sheet_name='BASE DATOS INE',
                         dtype={'COD_INE': str, 'Ud pob': str})

# Supongamos que tus dataframes se llaman df1 y df2
df1 = faltan
df2 = base_ine

df1_cop=df1.copy()

faltan2['COD_INE_MATCH'] = np.nan
faltan2['UD_POB_MATCH'] = np.nan
df1 = df1.drop('Unnamed: 0', axis=1)

df1 = pd.concat([df1, faltan2], ignore_index=True)

# Vamos a convertir los valores de las columnas POST_CODE y COD_INE a string
df1['POST_CODE1'] = df1['POST_CODE1'].astype(str)
df2['COD_INE'] = df2['COD_INE'].astype(str)

# Creamos una función para buscar la coincidencia de los dos primeros dígitos y un 80% de similitud en las ciudades
def find_cod_ine(row):
    post_code = row['POST_CODE1']
    city1 = row['CITY1']
    
    # Filtramos el df2 por los dos primeros dígitos
    df2_filtered = df2[df2['COD_INE'].str.startswith(post_code[:2])]
    
    # Si no hay coincidencias en los dos primeros dígitos, retornamos NaN
    if df2_filtered.empty:
        return float('nan')
    
    for index, row in df2_filtered.iterrows():
        if fuzz.ratio(city1.lower(), row['Ud pob'].lower()) >= 80:
            return row['COD_INE'], row['Ud pob']

    return float('nan'), float('nan')

# Aplicamos la función a cada fila de df1
df1[['COD_INE_MATCH', 'UD_POB_MATCH']] = df1.apply(
    lambda row: pd.Series(find_cod_ine(row)), axis=1)

df1_terminado=df1.loc[df1['COD_INE_MATCH'].notna()]
df1_terminado2=df1.loc[df1['COD_INE_MATCH'].notna()]
df1_terminado3=df1.loc[df1['COD_INE_MATCH'].notna()]
df1_terminado4=df1.loc[df1['COD_INE_MATCH'].notna()]

df1_terminado5= pd.concat([df1_terminado, df1_terminado2, df1_terminado3, df1_terminado4], ignore_index=True)

df1_terminado6=df1

df1=df1[df1['COD_INE_MATCH'].isna()]

len(df1)
df1_terminado6.to_excel('ruta_1/CP_INE/CP_INE_py_6.xlsx')
len(df1_terminado)

faltan = pd.read_excel('ruta_1/CP_INE/CP_INE_py_6.xlsx',dtype=str)
faltan['CITY1_2']=faltan['CITY1']

faltan['CITY1_2'] = faltan['CITY1_2'].str.replace("-", " ")
# faltan['CITY1_2'] = faltan['CITY1_2'].str.replace(",", " ")
faltan['CITY1_2'] = faltan['CITY1_2'].str.replace("(", " ")
faltan['CITY1_2'] = faltan['CITY1_2'].str.replace(")", " ")

def check_city_match(row):
    city1 = row['CITY1']
    pueblo_base_date = row['pueblo_base_date']

    # Comprobamos si alguno de los valores es NaN
    if pd.isnull(city1) or pd.isnull(pueblo_base_date):
        return 'ERROR'
    # Calculamos el porcentaje de coincidencia entre los dos nombres de la ciudad
    elif fuzz.ratio(city1.lower(), pueblo_base_date.lower()) >= 50:
        return 1
    else:
        return 0

# Aplicamos la función a cada fila del dataframe
faltan['city_match'] = faltan.apply(check_city_match, axis=1)

# Creamos un DataFrame solo con las filas donde 'city_match' es 1
df_match = faltan.loc[faltan['city_match'] == 1]

# Creamos otro DataFrame con las filas restantes
df_no_match = faltan.loc[faltan['city_match'] != 1]

df_match.to_excel('ruta_1/CP_INE/df_match.xlsx')
df_no_match.to_excel('ruta_1/CP_INE/df_NO_match.xlsx')

faltan = pd.read_excel('ruta_1/CP_INE/df_NO_match.xlsx',dtype=str)
datos_ine = pd.read_excel('ruta_1/CP_INE/CIS_2021_20230407.xlsx',sheet_name="BASE DATOS INE",dtype=str)

def match_name(name, list_names, min_score=0):
    max_score = -1
    max_name = ""
    for name2 in list_names:
        score = fuzz.ratio(name, name2)
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

dict_list = []

for name in tqdm(faltan['CITY1_2']):
    match = match_name(name, datos_ine['Ud pob'], 50)
    dict_ = {}
    dict_.update({"CITY1_2" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
print(merge_table)

datos_ine_list = datos_ine[['Ud pob', 'COD_INE']].values.tolist()

def match_name(row, list_names_codes, min_score=0):
    name = row['CITY1_2']
    code = row['ine_base_date']
    max_score = -1
    max_name = ""
    max_code = ""
    for name2, code2 in list_names_codes:
        score = fuzz.ratio(name, name2)
        if (score > min_score) & (score > max_score) & (code == code2):
            max_name = name2
            max_score = score
            max_code = code2
    row['match_name'] = max_name
    row['score'] = max_score
    row['match_code'] = max_code
    row['Ud pob'] = max_name
    row['COD_INE'] = max_code
    return row

faltan_copy = faltan.copy()

tqdm.pandas()

faltan_copy = faltan_copy.progress_apply(lambda row: match_name(row, datos_ine_list, 50), axis=1)

print(faltan_copy)

faltan_copy.to_excel('ruta_1/CP_INE/faltan_restantes.xlsx')